## Runtime and VPM impact of options of LRTSAC

File used to create figures 6 to 9 of the paper.

In [ ]:
import single_data_loader
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np

Glob path to the output files of the experiment:

In [ ]:
file_glob = "/home/riuclement/Documents/RANSAC-benchmark/ipol_experiment/time_exp/*/{}_{}_TimeMetrics.txt"

Number of datasets and name of the datasets:

In [ ]:
datasize = {"hom": 10, "fun": 11, "ess": 6}
datanames = ["hom", "fun", "ess"]

In [ ]:
generic_labels = [
    "LRT with bailout, with update of T, with reduce Sigma",
    "LRT with bailout, without update of T, with reduce Sigma",
    "LRT without bailout, with update of T, with reduce Sigma",
    "LRT without bailout, without update of T, with reduce Sigma",
    "LRT without bailout, without update of T, without reduce Sigma",
]

Loading of the data:

In [ ]:
time_loader = single_data_loader.Time_Data_Loader(
    file_glob,
    datasize,
    datanames,
    generic_labels,
    5
)

In [ ]:
time_loader.load_data()

In [ ]:
def create_ticks(datasize):
    x_names = [
        "H{}".format(i) for i in range(1, datasize["hom"] + 1)
    ] + [
        "F{}".format(i) for i in range(1, datasize["fun"] + 1)
    ] + [
        "E{}".format(i) for i in range(1, datasize["ess"] + 1)
    ]
    x_ticks = np.arange(len(x_names))
    return x_ticks, x_names

def plot_TimeMetric(metric_index, values, plot_name, plot=False, save=False):
    x_ticks, x_names = create_ticks(values._datasize)

    plt.figure(figsize=(10, 7))
    plt.title("{} across USAC datasets".format(plot_name))
    plt.xticks(x_ticks, x_names)
    for i in range(values._mean.shape[1]):
        if plot:
            plt.step(x_ticks, values._ratio[metric_index, i, :], label=values._labels[i], alpha=0.8, where="mid")
        else:
            plt.bar(x_ticks, values._ratio[metric_index, i, :], label=values._labels[i], alpha=1 / len(values._numberAlgo))
    plt.legend()
    plt.xlabel("Dataset name")
    plt.ylabel(plot_name)
    plt.tight_layout()
    if save:
        plt.savefig(save)
    else:
        plt.show()

Plotting of the runtime and vpm ratio:

In [ ]:
plot_TimeMetric(2,
                time_loader,
                "Runtime divided by runtime without any options",
                True,
                save=False)#"/home/riuclement/Documents/RANSAC-benchmark/ipol_experiment/qs_exp/all_time_ratio_lrt.pdf")

In [ ]:
plot_TimeMetric(1,
                time_loader,
                "VPM as a ratio of total number of points",
                True)#, save="/home/riuclement/Documents/RANSAC-benchmark/ipol_experiment/qs_exp/vpm_ratio_lrt.pdf")

In [ ]:
def plot_TimeMetric_multiple_bar(values, plot_names, save=False):
    x_ticks, x_names = create_ticks(values._datasize)

    begin = 0
    for datasetName, nData in values._datasize.items():
        plt.figure(figsize=(max(nData, 7), 7))
        plt.title("Runtime divided by runtime without any options for USAC {} image pairs.".format(plot_names[datasetName]))
        plt.xticks(x_ticks[begin:begin+nData] * 1.1, x_names[begin:begin+nData])
        for i in range(values._numberAlgo):
            plt.bar((x_ticks[begin:begin+nData]) * 1.1 - 2 / values._numberAlgo + i / values._numberAlgo, 
                    values._ratio[2, i, begin:begin+nData], 
                    1 / values._numberAlgo, 
                    label=values._labels[i],
                    align="center")
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1))
        plt.xlabel("Dataset name")
        plt.ylabel("Runtime divided by runtime without any options (log scale)")
        plt.tight_layout()
        plt.yscale('log')
        if save:
            plt.savefig(save.format(datasetName))
        else:
            plt.show()
        begin+=nData

Plot of the runtime ratio for all options:

In [ ]:
plot_TimeMetric_multiple_bar(time_loader,
                             {"hom": "homography", "fun": "fundamental matrix", "ess": "essential matrix"},
                             save=False)#"/home/riuclement/Documents/RANSAC-benchmark/ipol_experiment/qs_exp/all_time_ratio_lrt_{}.pdf")

In [ ]:
def plot_TimeMetric_dual(values, plot_name, ylabels, semilog=False, save=False):
    x_ticks, x_names = create_ticks(values._datasize)
    alpha = 0.6
    
    t_ratio = values._mean[2, 0:1, :] / values._mean[2, 2:3, :]
    vpm_ratio = values._mean[1, 0:1, :] / values._mean[1, 2:3, :]
    
    fig, ax1 = plt.subplots(figsize=(10, 7))
    fig.suptitle("{} across USAC datasets".format(plot_name))
    ax1.set_xticks(x_ticks)
    ax1.set_xticklabels(x_names)
    ax1.set_xlabel("Dataset names")
    
    line1 = ax1.bar(x_ticks, t_ratio[0, :], color='b', alpha=alpha)
    ax1.set_ylabel(ylabels[0], color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1.set_ylim(0, t_ratio[0, :].max() * 1.05)
    
    ax2 = ax1.twinx()

    line2 = ax2.bar(x_ticks, vpm_ratio[0, :], color='r', alpha=alpha)
    ax2.set_ylabel(ylabels[1], color='r')
    ax2.tick_params(axis='y', labelcolor='r')
    ax2.set_ylim(0, t_ratio[0, :].max() * 1.05)
    

    fig.tight_layout()
    if save:
        plt.savefig(save)
    else:
        plt.show()

Compare runtime and vpm with and without bailout:

In [ ]:
plot_TimeMetric_dual(time_loader,
                    "Runtime and VPM ratios",
                    ["LRT runtime with bailout / without bailout", "LRT VPM / total number of points"],
                    save=False)#"/home/riuclement/Documents/RANSAC-benchmark/ipol_experiment/qs_exp/new_time_vpm_ratio_lrt.pdf")